In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Hyperparameter Tuning for XGBoost

## Overview


This tutorial demonstrates how to use Vertex AI to perform Hyperparameter Tuning for XGBoost model.


This tutorial uses the following Google Cloud ML services:

- `Vertex AI Training`
- `Vertex AI Hyperparameter Tuning`
- `Vertex AI Vizier`
- `Vertex AI Model` resource

The steps performed include:

- Training using a Python package.
- Report accuracy when hyperparameter tuning is done.
- Save the model artifacts to Cloud Storage using GCSFuse.
- Create a `Vertex AI Model` resource.

### Dataset

The dataset used for this tutorial is the [Iris dataset](https://www.tensorflow.org/datasets/catalog/iris). This dataset does not require any feature engineering. The version of the dataset in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of Iris flower species from a class of three species: setosa, virginica, or versicolor.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


## Installations

Install the following packages to execute this notebook.

In [3]:
import os

USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [5]:
import os
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Before you begin

### Set up your Google Cloud project

The following steps are required


1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). 


2. Enter your project ID in the cell below. Then run the cell to make sure the


#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [2]:
PROJECT_ID = 'ds-training-380514'  # @param {type:"string"}

In [3]:
! gcloud config set project $PROJECT_ID --quiet

Updated property [core/project].


In [4]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [5]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

In [6]:
BUCKET_NAME = 'lending-club-xgb-model01'
BUCKET_URI = "gs://" + BUCKET_NAME

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [7]:
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://lending-club-xgb-model01/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [8]:
! gsutil ls -al $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [9]:
import google.cloud.aiplatform as aip

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [10]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Set pre-built containers

Set the pre-built Docker container image for training and prediction.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [65]:
TRAIN_VERSION = "xgboost-cpu.1-1"
DEPLOY_VERSION = "xgboost-cpu.1-1"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

#### Set machine type

Next, set the machine type to use for training.

- Set the variable `TRAIN_COMPUTE` to configure  the compute resources for the VMs you will use for for training.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: The following is not supported for training:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs

In [12]:

MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

Train machine type n1-standard-4


## Introduction to XGBoost training

Once you have trained a XGBoost model, you will want to save it at a Cloud Storage location, so it can subsequently be uploaded to a `Vertex AI Model` resource.
The XGBoost package does not have support to save the model to a Cloud Storage location. Instead, you will do the following steps to save to a Cloud Storage location.

1. Save the in-memory model to the local filesystem (e.g., model.bst).
2. Use gsutil to copy the local copy to the specified Cloud Storage location.

## Vertex AI Hyperparameter Tuning service

The following example demonstrates how to setup, execute and evaluate trials using the Vertex AI Hyperparameter Tuning service with `Vizier` search algorithm.

Learn more about [Overview of hyperparameter tuning](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)

### Examine the training package

#### Package layout

Before you start the training, you will look at how a Python package is assembled for a custom training job. When unarchived, the package contains the following directory/file layout.

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

The files `setup.cfg` and `setup.py` are the instructions for installing the package into the operating environment of the Docker image.

The file `trainer/task.py` is the Python script for executing the custom training job. *Note*, when we referred to it in the worker pool specification, we replace the directory slash with a dot (`trainer.task`) and dropped the file suffix (`.py`).

#### Package Assembly

In the following cells, you will assemble the training package.

In [13]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'cloudml-hypertune',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Iris tabular classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Create the task script for the Python training package

Next, you create the `task.py` script for driving the training package. Some noteable steps include:

- Command-line arguments:
    - `model-dir`: The location to save the trained model. When using Vertex AI custom training, the location will be specified in the environment variable: `AIP_MODEL_DIR`,
    - `dataset_data_url`: The location of the training data to download.
    - `dataset_labels_url`: The location of the training labels to download.
    - `boost-rounds`: Tunable hyperparameter
- Data preprocessing (`get_data()`):
    - Download the dataset and split into training and test.
- Training (`train_model()`):
    - Trains the model
- Evaluation (`evaluate_model()`):
    - Evaluates the model.
    - If hyperparameter tuning, reports the metric for accuracy.
- Model artifact saving
    - Saves the model artifacts and evaluation metrics where the Cloud Storage location specified by `model-dir`.

In [14]:
%%writefile custom/trainer/task.py
import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb
import hypertune
import argparse
import logging

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument("--dataset-data-url", dest="dataset_data_url",
                    type=str, help="Download url for the training data.")
parser.add_argument("--dataset-labels-url", dest="dataset_labels_url",
                    type=str, help="Download url for the training data labels.")
parser.add_argument("--boost-rounds", dest="boost_rounds",
                    default=20, type=int, help="Number of boosted rounds")
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

def get_data():
    logging.info("Downloading training data and labelsfrom: {}, {}".format(args.dataset_data_url, args.dataset_labels_url))
    # gsutil outputs everything to stderr so we need to divert it to stdout.
    subprocess.check_call(['gsutil', 'cp', args.dataset_data_url, 'data.csv'], stderr=sys.stdout)
    # gsutil outputs everything to stderr so we need to divert it to stdout.
    subprocess.check_call(['gsutil', 'cp', args.dataset_labels_url, 'labels.csv'], stderr=sys.stdout)


    # Load data into pandas, then use `.values` to get NumPy arrays
    data = pd.read_csv('data.csv').values
    labels = pd.read_csv('labels.csv').values

    # Convert one-column 2D array into 1D array for use with XGBoost
    labels = labels.reshape((labels.size,))

    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=7)

    # Load data into DMatrix object
    dtrain = xgb.DMatrix(train_data, label=train_labels)
    return dtrain, test_data, test_labels

def train_model(dtrain):
    logging.info("Start training ...")
    # Train XGBoost model
    model = xgb.train({}, dtrain, num_boost_round=args.boost_rounds)
    logging.info("Training completed")
    return model

def evaluate_model(model, test_data, test_labels):
    dtest = xgb.DMatrix(test_data)
    pred = model.predict(dtest)
    predictions = [round(value) for value in pred]
    # evaluate predictions
    accuracy = accuracy_score(test_labels, predictions)
    logging.info(f"Evaluation completed with model accuracy: {accuracy}")

    # report metric for hyperparameter tuning
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='accuracy',
        metric_value=accuracy
    )
    return accuracy


dtrain, test_data, test_labels = get_data()
model = train_model(dtrain)
accuracy = evaluate_model(model, test_data, test_labels)

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

# Export the classifier to a file
gcs_model_path = os.path.join(args.model_dir, 'model.bst')
logging.info("Saving model artifacts to {}". format(gcs_model_path))
model.save_model(gcs_model_path)

logging.info("Saving metrics to {}/metrics.json". format(args.model_dir))
gcs_metrics_path = os.path.join(args.model_dir, 'metrics.json')
with open(gcs_metrics_path, "w") as f:
    f.write(f"{'accuracy: {accuracy}'}")

Writing custom/trainer/task.py


#### Store training script on your Cloud Storage bucket

Next, you package the training folder into a compressed tar ball, and then store it in your Cloud Storage bucket.

In [18]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_iris.tar.gz

custom/
custom/PKG-INFO
custom/setup.cfg
custom/setup.py
custom/trainer/
custom/trainer/task.py
custom/trainer/__init__.py
custom/README.md
Copying file://custom.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.8 KiB/  1.8 KiB]                                                
Operation completed over 1 objects/1.8 KiB.                                      


### Prepare your machine specification

Now define the machine specification for your custom hyperparameter tuning job. This tells Vertex what type of machine instance to provision for the hyperparameter tuning.
  - `machine_type`: The type of GCP instance to provision -- e.g., n1-standard-8.
  - `accelerator_type`: The type, if any, of hardware accelerator. In this tutorial if you previously set the variable `TRAIN_GPU != None`, you are using a GPU; otherwise you will use a CPU.
  - `accelerator_count`: The number of accelerators.

In [19]:

machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### Prepare your disk specification

(optional) Now define the disk specification for your custom hyperparameter tuning job. This tells Vertex what type and size of disk to provision in each machine instance for the hyperparameter tuning.

  - `boot_disk_type`: Either SSD or Standard. SSD is faster, and Standard is less expensive. Defaults to SSD.
  - `boot_disk_size_gb`: Size of disk in GB.

In [20]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

In [87]:
DATASET_DIR = "gs://cloud-samples-data/ai-platform/iris"

In [88]:
label_uri= DATASET_DIR + "/iris_target.csv"

In [90]:
df_label=pd.read_csv(label_uri)
df_label.iloc[:,0].unique()

array([0, 1, 2])

### Prepare your command-line arguments

Now define the command-line arguments for your custom training container:

- `args`: The command-line arguments to pass to the executable that is set as the entry point into the container.
  - `--model-dir` : For our demonstrations, we use this command-line argument to specify where to store the model artifacts.
      - direct: You pass the Cloud Storage location as a command line argument to your training script (set variable `DIRECT = True`), or
      - indirect: The service passes the Cloud Storage location as the environment variable `AIP_MODEL_DIR` to your training script (set variable `DIRECT = False`). In this case, you tell the service the model artifact location in the job specification.
  - `--dataset-data-url`: The location of the training data to download.
  - `--dataset-labels-url`: The location of the training labels to download.
  - `--boost-rounds`: Tunable hyperparameter.

In [21]:
MODEL_DIR = "{}/aiplatform-custom-job-{}".format(BUCKET_URI, TIMESTAMP)
DATASET_DIR = "gs://cloud-samples-data/ai-platform/iris"


CMDARGS = [
    "--dataset-data-url=" + DATASET_DIR + "/iris_data.csv",
    "--dataset-labels-url=" + DATASET_DIR + "/iris_target.csv",
]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_URI + "/trainer_iris.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

## Create a custom job

Use the class `CustomJob` to create a custom job, such as for hyperparameter tuning, with the following parameters:

- `display_name`: A human readable name for the custom job.
- `worker_pool_specs`: The specification for the corresponding VM instances.
- `base_output_dir`: The Cloud Storage location for storing the model artifacts.

In [24]:
job = aip.CustomJob(
    display_name="iris_" + TIMESTAMP,
    worker_pool_specs=worker_pool_spec,
    base_output_dir=MODEL_DIR,
)

## Create a hyperparameter tuning job

Use the class `HyperparameterTuningJob` to create a hyperparameter tuning job, with the following parameters:

- `display_name`: A human readable name for the custom job.
- `custom_job`: The worker pool spec from this custom job applies to the CustomJobs created in all the trials.
- `metrics_spec`: The metrics to optimize. The dictionary key is the metric_id, which is reported by your training job, and the dictionary value is the optimization goal of the metric('minimize' or 'maximize').
- `parameter_spec`: The parameters to optimize. The dictionary key is the metric_id, which is passed into your training job as a command line key word argument, and the dictionary value is the parameter specification of the metric.
- `search_algorithm`: The search algorithm to use: `grid`, `random` and `None`. If `None` is specified, the `Vizier` service (Bayesian) is used.
- `max_trial_count`: The maximum number of trials to perform.

In [25]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aip.HyperparameterTuningJob(
    display_name="iris_" + TIMESTAMP,
    custom_job=job,
    metric_spec={
        "accuracy": "maximize",
    },
    parameter_spec={
        "boost-rounds": hpt.IntegerParameterSpec(min=10, max=100, scale="linear"),
    },
    search_algorithm=None,
    max_trial_count=6,
    parallel_trial_count=2,
)

## Run the hyperparameter tuning job

Use the `run()` method to execute the hyperparameter tuning job.

In [26]:
hpt_job.run()

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/354621994428/locations/us-central1/hyperparameterTuningJobs/4656744417250508800
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/354621994428/locations/us-central1/hyperparameterTuningJobs/4656744417250508800')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4656744417250508800?project=354621994428
HyperparameterTuningJob projects/354621994428/locations/us-central1/hyperparameterTuningJobs/4656744417250508800 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/354621994428/locations/us-central1/hyperparameterTuningJobs/4656744417250508800 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/354621994428/locations/us-central1/hyperparameterTuningJobs/4656744417250508800 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJ

### Display the hyperparameter tuning job trial results

After the hyperparameter tuning job has completed, the property `trials` will return the results for each trial.

In [27]:
print(hpt_job.trials)

[id: "1"
state: SUCCEEDED
parameters {
  parameter_id: "boost-rounds"
  value {
    number_value: 55.0
  }
}
final_measurement {
  step_count: 1
  metrics {
    metric_id: "accuracy"
    value: 0.8666666666666667
  }
}
start_time {
  seconds: 1680679064
  nanos: 80007852
}
end_time {
  seconds: 1680679197
}
, id: "2"
state: SUCCEEDED
parameters {
  parameter_id: "boost-rounds"
  value {
    number_value: 35.0
  }
}
final_measurement {
  step_count: 1
  metrics {
    metric_id: "accuracy"
    value: 0.8666666666666667
  }
}
start_time {
  seconds: 1680679064
  nanos: 80111896
}
end_time {
  seconds: 1680679210
}
, id: "3"
state: SUCCEEDED
parameters {
  parameter_id: "boost-rounds"
  value {
    number_value: 79.0
  }
}
final_measurement {
  step_count: 1
  metrics {
    metric_id: "accuracy"
    value: 0.8666666666666667
  }
}
start_time {
  seconds: 1680679586
  nanos: 127397975
}
end_time {
  seconds: 1680679712
}
, id: "4"
state: SUCCEEDED
parameters {
  parameter_id: "boost-rounds"

### Best trial

Now look at which trial was the best:

In [28]:
best = (None, None, None, 0.0)
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

print(best)

('1', 55.0, None, 0.8666666666666667)


### Get the Best Model

If you used the method of having the service tell the tuning script where to save the model artifacts (`DIRECT = False`), then the model artifacts for the best model are saved at:

    MODEL_DIR/<best_trial_id>/model

In [29]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

! gsutil ls {BEST_MODEL_DIR}

gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/1/model/
gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/1/model/metrics.json
gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/1/model/model.bst


# Deploy the model to Vertex AI Prediction

In [66]:
MODEL_NAME = 'xgb_classifier_2'
SERVING_CONTAINER_IMAGE_URI = DEPLOY_IMAGE
SERVING_MACHINE_TYPE = "n1-standard-2"


In [54]:
from google.cloud import aiplatform

### Uploading the trained model

In [67]:
uploaded_model = aiplatform.Model.upload(
    display_name=MODEL_NAME,
    artifact_uri=BEST_MODEL_DIR,
    serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
)

Creating Model
Create Model backing LRO: projects/354621994428/locations/us-central1/models/3676355665934155776/operations/1789059143861010432
Model created. Resource name: projects/354621994428/locations/us-central1/models/3676355665934155776@1
To use this Model in another session:
model = aiplatform.Model('projects/354621994428/locations/us-central1/models/3676355665934155776@1')


In [95]:

import pandas as pd
TARGET_DIR = "gs://cloud-samples-data/ai-platform/iris/iris_target.csv"


df_target = pd.read_csv(TARGET_DIR).iloc[[0,1,66,88,145,148],:]
df_target

,0
0,0
1,0
66,1
88,1
145,2
148,2


In [96]:
import pandas as pd
DATASET_DIR = "gs://cloud-samples-data/ai-platform/iris/iris_data.csv"


df_inference = pd.read_csv(DATASET_DIR).iloc[:10,:]
df_inference = pd.read_csv(DATASET_DIR).iloc[[0,1,66,88,145,148],:]
df_inference.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
66,5.6,3.0,4.5,1.5
88,5.6,3.0,4.1,1.3
145,6.7,3.0,5.2,2.3


In [124]:
df_inference.index.tolist()

[0, 1, 66, 88, 145, 148]

In [98]:
import xgboost as xgb
dtest = xgb.DMatrix(df_inference)

In [113]:
df_inference.to_csv("gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/inference_data.csv",index=False)

In [114]:
# parameters
LOCATION = "us-central1"
PROJECT = "ds-training-380514"
BATCH_JOB_NAME = "lending_batch"
MODEL_ID = "projects/354621994428/locations/us-central1/models/3676355665934155776@1"
URI = "gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/inference_data.csv"
OUTPUT_URI_PREFIX = "gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/batch_predict_output"
MACHINE_TYPE = SERVING_MACHINE_TYPE
STARTING_REPLICA_COUNT = 1
MAX_REPLICA_COUNT = 10  # default
GENERATE_EXPLANATION = "false"  # default value

In [115]:
request_body = {
    "displayName": BATCH_JOB_NAME,
    "model": MODEL_ID,
    "inputConfig": {
        "instancesFormat": "csv",
        "gcsSource": {
            "uris": [
                URI
            ]
        },
    },
    "outputConfig": {
        "predictionsFormat": "jsonl",
        "gcsDestination": {
            "outputUriPrefix": OUTPUT_URI_PREFIX
        }
    },
    "dedicatedResources": {
        "machineSpec": {
            "machineType": MACHINE_TYPE,
            "acceleratorCount": "0"
        },
        "startingReplicaCount": STARTING_REPLICA_COUNT,
        "maxReplicaCount": MAX_REPLICA_COUNT
    },
    "generateExplanation": GENERATE_EXPLANATION
}

In [116]:
request_body

{'displayName': 'lending_batch',
 'model': 'projects/354621994428/locations/us-central1/models/3676355665934155776@1',
 'inputConfig': {'instancesFormat': 'csv',
  'gcsSource': {'uris': ['gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/inference_data.csv']}},
 'outputConfig': {'predictionsFormat': 'jsonl',
  'gcsDestination': {'outputUriPrefix': 'gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/batch_predict_output'}},
 'dedicatedResources': {'machineSpec': {'machineType': 'n1-standard-2',
   'acceleratorCount': '0'},
  'startingReplicaCount': 1,
  'maxReplicaCount': 10},
 'generateExplanation': 'false'}

In [117]:
import json
with open("requestay.json", "w") as outfile:
    outfile.write(json.dumps(request_body))

In [118]:
region='us-central1'
project = 'ds-training-380514'

In [119]:
api_endpoint = f"https://{region}-aiplatform.googleapis.com/v1/projects/{project}/locations/{region}/batchPredictionJobs"

In [120]:
!curl -X POST \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "Content-Type: application/json; charset=utf-8" \
    -d @requestay.json \
    {api_endpoint}

{
  "name": "projects/354621994428/locations/us-central1/batchPredictionJobs/9093775212628934656",
  "displayName": "lending_batch",
  "model": "projects/354621994428/locations/us-central1/models/3676355665934155776",
  "inputConfig": {
    "instancesFormat": "csv",
    "gcsSource": {
      "uris": [
        "gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/inference_data.csv"
      ]
    }
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/batch_predict_output"
    }
  },
  "dedicatedResources": {
    "machineSpec": {
      "machineType": "n1-standard-2"
    },
    "startingReplicaCount": 1,
    "maxReplicaCount": 10
  },
  "manualBatchTuningParameters": {},
  "state": "JOB_STATE_PENDING",
  "createTime": "2023-04-05T15:37:05.055976Z",
  "updateTime": "2023-04-05T15:37:05.055976Z",
  "modelVersionId": "1"
}


In [121]:
import pandas as pd 
file_path = "gs://lending-club-xgb-model01/aiplatform-custom-job-20230405071155/batch_predict_output/prediction-xgb_classifier_2-2023_04_05T06_19_32_065Z/prediction.results-00000-of-00002"
jsonObj = pd.read_json(path_or_buf=file_path, lines=True)
jsonObj

,instance,prediction
0,"[5.1, 3.5, 1.4, 0.2]",0.000102
1,"[4.9, 3.1, 1.5, 0.1]",0.000102
2,"[5.0, 3.6, 1.4, 0.2]",0.000102
3,"[4.6, 3.4, 1.4, 0.30000000000000004]",0.000102
4,"[5.0, 3.4, 1.5, 0.2]",0.000102
5,"[4.7, 3.2, 1.3, 0.2]",0.000102
6,"[5.4, 3.9, 1.7000000000000002, 0.4]",0.000102


### Deploying the uploaded model

In [83]:
endpoint = uploaded_model.deploy(
    machine_type=SERVING_MACHINE_TYPE,
    accelerator_type=None,
    accelerator_count=None,
)

Creating Endpoint
Create Endpoint backing LRO: projects/354621994428/locations/us-central1/endpoints/7409490112326664192/operations/97957483783389184
Endpoint created. Resource name: projects/354621994428/locations/us-central1/endpoints/7409490112326664192
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/354621994428/locations/us-central1/endpoints/7409490112326664192')
Deploying model to Endpoint : projects/354621994428/locations/us-central1/endpoints/7409490112326664192
Deploy Endpoint model backing LRO: projects/354621994428/locations/us-central1/endpoints/7409490112326664192/operations/4975355880225636352
Endpoint model deployed. Resource name: projects/354621994428/locations/us-central1/endpoints/7409490112326664192


### Serve predictions

In [134]:
for i in range(df_inference.shape[0]):
    instance = df_inference.iloc[i,:].values.tolist()
    response= endpoint.predict([instance])
    #print(endpoint.predict([instance]))
    print(response.predictions)

[0.0001016855239868164]
[0.0001016855239868164]
[0.9999709725379944]
[0.9999709725379944]
[2.000031232833862]
[1.999940752983093]


In [135]:
from typing import List, Dict

def predict_tabular_classification(
    project: str,
    location: str,
    endpoint_name: str,
    instances: List[Dict],
):
    """
    Args
        project: Your project ID or project number.
        location: Region where Endpoint is located. For example, 'us-central1'.
        endpoint_name: A fully qualified endpoint name or endpoint ID. Example: "projects/123/locations/us-central1/endpoints/456" or
               "456" when project and location are initialized or passed.
        instances: A list of one or more instances (examples) to return a prediction for.
    """
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint_name)

    response = endpoint.predict(instances=instances)

    for prediction_ in response.predictions:
        print(prediction_)

In [138]:
for i in range(df_inference.shape[0]):
    instance = df_inference.iloc[i,:].values.tolist()
    inference_results = predict_tabular_classification(PROJECT_ID, REGION, 'projects/354621994428/locations/us-central1/endpoints/7409490112326664192', [instance])


0.0001016855239868164
0.0001016855239868164
0.9999709725379944
0.9999709725379944
2.000031232833862
1.999940752983093


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Custom Job (Custome Training job is remove in previous step)
- Cloud Storage Bucket

In [ ]:
delete_bucket = False

if delete_bucket :
    ! gsutil rm -r $BUCKET_URI